# Installing prerequisites

We are going to configure this colab instance by:

* downloading spark-3.2.0
* downloading a binary distribution of quarkus nessie

We are going to access web uis via tunnels provided by ngrok (register with your github account or google account and get your auth token)

replace `THE_AUTH_TOKEN_FOR_NGROK` with your actual auth token

In [1]:
%%shell
mkdir -p build
cd build
echo "Installing SPARK"
wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
tar xf spark-3.2.0-bin-hadoop3.2.tgz
echo "Installing FINDSPARK"
pip -q install findspark
echo "Installing NESSIE"
wget -q https://github.com/andrea-rockt/colab-notebooks/raw/main/data/nessie-quarkus-0.18.0.bin
chmod +x nessie-quarkus-0.18.0.bin
wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
tar xf ngrok-stable-linux-amd64.tgz

./ngrok authtoken THE_AUTH_TOKEN_FOR_NGROK

Installing SPARK
Installing FINDSPARK
Installing NESSIE
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


We will start nessie as a background process nessie will serve its web UI at localhost:19120

nessie will use in memory persistence so everything we do will be ephemeral

In [2]:
import os
os.system("/content/build/nessie-quarkus-0.18.0.bin &")

0

In [3]:
import findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/build/spark-3.2.0-bin-hadoop3.2"
findspark.init()
from pyspark.sql import SparkSession
spark= SparkSession \
       .builder \
       .appName("Our First Spark example") \
       .getOrCreate()
spark